<a href="https://colab.research.google.com/github/D3xter1922/MHI_exp/blob/main/Roberta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
device='cuda'

In [1]:
!pip install transformers

     |████████████████████████████████| 3.8 MB 5.3 MB/s 
     |████████████████████████████████| 596 kB 45.8 MB/s 
     |████████████████████████████████| 67 kB 5.1 MB/s 
     |████████████████████████████████| 6.5 MB 32.6 MB/s 
     |████████████████████████████████| 895 kB 41.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
!pip install modin[ray]

     |████████████████████████████████| 761 kB 5.1 MB/s 
     |████████████████████████████████| 134 kB 49.8 MB/s 
     |████████████████████████████████| 52.7 MB 157 kB/s 
     |████████████████████████████████| 175 kB 48.9 MB/s 
     |████████████████████████████████| 4.1 MB 36.1 MB/s 
     |████████████████████████████████| 144 kB 49.4 MB/s 
     |████████████████████████████████| 3.0 MB 31.3 MB/s 
     |████████████████████████████████| 201 kB 44.2 MB/s 
     |████████████████████████████████| 65 kB 3.8 MB/s 
     |████████████████████████████████| 128 kB 48.6 MB/s 
     |████████████████████████████████| 82 kB 263 kB/s 
     |████████████████████████████████| 1.1 MB 40.2 MB/s 
     |████████████████████████████████| 94 kB 3.8 MB/s 
     |████████████████████████████████| 271 kB 46.9 MB/s 
     |████████████████████████████████| 85 kB 3.9 MB/s 
     |████████████████████████████████| 58 kB 5.8 MB/s 
  Created wheel for gpustat: filename=gpustat-1.0.0b1-py3-none-any.whl size=15979 s

In [4]:
import modin.pandas as pd
import os
import glob
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import seaborn as sns
import transformers
import json
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaModel, RobertaTokenizer
import logging
logging.basicConfig(level=logging.ERROR)

In [5]:
!cd '/content/drive/MyDrive/proofs'

In [6]:
os.getcwd()

'/content'

In [7]:

  
  
# use glob to get all the csv files 
# in the folder
path = '/content/drive/MyDrive/proofs/'
csv_files = glob.glob(os.path.join(path, "*.json"))
  
dataset = pd.DataFrame()
df=pd.DataFrame()
# loop over the list of csv files
for f in csv_files:
      
    # read the csv file
    
      
    # print the location and filename
    
    df2 = df
    df = pd.read_json(f, lines=True)
    dataset = pd.concat([df, dataset])
display(dataset)

To remove this warning, run the following python code before doing dataframe operations:

    import ray
    ray.init()

(deploy_ray_func pid=445) FutureWarning: the 'numpy' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'numpy'
(deploy_ray_func pid=441) FutureWarning: the 'numpy' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'numpy'


,P,H,label,goal,proof
0,Jonathan asked Steven for 50 dollars. Steven h...,Steven did not have 50 dollars.,entailment,NaN,NaN
1,Matthew asked Annie for 40 dollars. Annie had 30.,Annie did not have 40 dollars.,entailment,NaN,NaN
2,Martin asked Jack for 50 dollars. Jack had 10.,Jack did not have 50 dollars.,entailment,NaN,NaN
3,Kelly asked Stephen for 200 dollars. Stephen h...,Stephen did not have 200 dollars.,entailment,NaN,NaN
4,Kevin asked Jean for 1000 dollars. Jean had 500.,Jean did not have 1000 dollars.,entailment,NaN,NaN
...,...,...,...,...,...
995,Arthur is running and so is Fiona.,Fiona is not running.,contradiction,"is_continuous_verb(Fiona,running)","[is_continuous_verb(Fiona,running)]"
996,Carol is dancing and so is Melissa.,Melissa is not dancing.,contradiction,"is_continuous_verb(Melissa,dancing)","[is_continuous_verb(Melissa,dancing)]"
997,Marilyn is exercising and so is Ron.,Ron is not exercising.,contradiction,"is_continuous_verb(Ron,exercising)","[is_continuous_verb(Ron,exercising)]"
998,Ron is exercising and so is Kenneth.,Kenneth is not exercising.,contradiction,"is_continuous_verb(Kenneth,exercising)","[is_continuous_verb(Kenneth,exercising)]"


In [8]:
display(dataset)

,P,H,label,goal,proof
0,Jonathan asked Steven for 50 dollars. Steven h...,Steven did not have 50 dollars.,entailment,NaN,NaN
1,Matthew asked Annie for 40 dollars. Annie had 30.,Annie did not have 40 dollars.,entailment,NaN,NaN
2,Martin asked Jack for 50 dollars. Jack had 10.,Jack did not have 50 dollars.,entailment,NaN,NaN
3,Kelly asked Stephen for 200 dollars. Stephen h...,Stephen did not have 200 dollars.,entailment,NaN,NaN
4,Kevin asked Jean for 1000 dollars. Jean had 500.,Jean did not have 1000 dollars.,entailment,NaN,NaN
...,...,...,...,...,...
995,Arthur is running and so is Fiona.,Fiona is not running.,contradiction,"is_continuous_verb(Fiona,running)","[is_continuous_verb(Fiona,running)]"
996,Carol is dancing and so is Melissa.,Melissa is not dancing.,contradiction,"is_continuous_verb(Melissa,dancing)","[is_continuous_verb(Melissa,dancing)]"
997,Marilyn is exercising and so is Ron.,Ron is not exercising.,contradiction,"is_continuous_verb(Ron,exercising)","[is_continuous_verb(Ron,exercising)]"
998,Ron is exercising and so is Kenneth.,Kenneth is not exercising.,contradiction,"is_continuous_verb(Kenneth,exercising)","[is_continuous_verb(Kenneth,exercising)]"


In [9]:
from transformers import RobertaTokenizerFast
# Defining some key variables that will be used later on in the training
MAX_LEN = 256
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 4
# EPOCHS = 1
LEARNING_RATE = 1e-05
tokenizer = RobertaTokenizer.from_pretrained('roberta-large', truncation=True, do_lower_case=True)


Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/482 [00:00<?, ?B/s]

In [10]:
tokenizer.encode_plus("hi my name is dog. I am an animal. I like bone")

{'input_ids': [0, 3592, 127, 766, 16, 2335, 4, 38, 524, 41, 3477, 4, 38, 101, 9013, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [11]:
class NLI_Data(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.Phrase
        self.targets = self.data.label
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [12]:
dataset['Phrase'] = dataset['P']+dataset['H']
new_df = dataset[['Phrase', 'label']]
display(new_df)

,Phrase,label
0,Jonathan asked Steven for 50 dollars. Steven h...,entailment
1,Matthew asked Annie for 40 dollars. Annie had ...,entailment
2,Martin asked Jack for 50 dollars. Jack had 10....,entailment
3,Kelly asked Stephen for 200 dollars. Stephen h...,entailment
4,Kevin asked Jean for 1000 dollars. Jean had 50...,entailment
...,...,...
995,Arthur is running and so is Fiona.Fiona is not...,contradiction
996,Carol is dancing and so is Melissa.Melissa is ...,contradiction
997,Marilyn is exercising and so is Ron.Ron is not...,contradiction
998,Ron is exercising and so is Kenneth.Kenneth is...,contradiction


In [13]:
!pip install modin[ray]

In [41]:
from sklearn.model_selection import train_test_split
new_df['label'] = np.where(new_df['label'] == 'entailment', 0, new_df['label'])
new_df['label'] = np.where(new_df['label'] == 'neutral', 1, new_df['label'])
new_df['label'] = np.where(new_df['label'] == 'contradiction', 2, new_df['label'])
train_data, test_data = train_test_split(new_df, test_size=0.2)

display(train_data)
display(new_df)


print("FULL Dataset: {}".format(new_df.shape))
print("TRAIN Dataset: {}".format(train_data.shape))
print("TEST Dataset: {}".format(test_data.shape))

training_set = NLI_Data(train_data, tokenizer, MAX_LEN)
testing_set = NLI_Data(test_data, tokenizer, MAX_LEN)

,Phrase,label
464,Alice is eating a pear.Alice is eating a raspb...,2
483,Pamela lives in Greece.Pamela is helpful.,1
645,Cookie and pitcher lie on table. Carl asked fo...,2
415,Julia has 2 dictionaries.Julia has atleast 1 d...,0
13,Nicole is weaker than Jimmy. Jimmy is weaker t...,0
...,...,...
951,Ray reached office at 12 noon whereas Rose rea...,0
793,Ben has geology class then commerce class then...,2
764,None of the lipsticks are maroon in colour.Som...,2
589,All of the files are maroon in colour.None of ...,0


,Phrase,label
0,Jonathan asked Steven for 50 dollars. Steven h...,0
1,Matthew asked Annie for 40 dollars. Annie had ...,0
2,Martin asked Jack for 50 dollars. Jack had 10....,0
3,Kelly asked Stephen for 200 dollars. Stephen h...,0
4,Kevin asked Jean for 1000 dollars. Jean had 50...,0
...,...,...
995,Arthur is running and so is Fiona.Fiona is not...,2
996,Carol is dancing and so is Melissa.Melissa is ...,2
997,Marilyn is exercising and so is Ron.Ron is not...,2
998,Ron is exercising and so is Kenneth.Kenneth is...,2


FULL Dataset: (363000, 2)
TRAIN Dataset: (290400, 2)
TEST Dataset: (72600, 2)


In [43]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }


training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [46]:
class RobertaClass(torch.nn.Module):
    def __init__(self):
        super(RobertaClass, self).__init__()
        self.l1 = RobertaModel.from_pretrained("roberta-base")
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(768, 3)

    def forward(self, input_ids, attention_mask, token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

In [47]:
model = RobertaClass()
model.to('cuda')

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


RobertaClass(
  (l1): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), e

In [48]:
# Creating the loss function and optimizer
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [49]:
def calcuate_accuracy(preds, targets):
    n_correct = (preds==targets).sum().item()
    return n_correct

In [50]:
# Defining the training function on the 80% of the dataset for tuning the distilbert model

def train(epoch):
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    model.train()
    for _,data in tqdm(enumerate(training_loader, 0)):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long)

        outputs = model(ids, mask, token_type_ids)
        loss = loss_function(outputs, targets)
        tr_loss += loss.item()
        big_val, big_idx = torch.max(outputs.data, dim=1)
        n_correct += calcuate_accuracy(big_idx, targets)

        nb_tr_steps += 1
        nb_tr_examples+=targets.size(0)
        
        if _%5000==0:
            loss_step = tr_loss/nb_tr_steps
            accu_step = (n_correct*100)/nb_tr_examples 
            print(f"Training Loss per 5000 steps: {loss_step}")
            print(f"Training Accuracy per 5000 steps: {accu_step}")

        optimizer.zero_grad()
        loss.backward()
        # # When using GPU
        optimizer.step()

    print(f'The Total Accuracy for Epoch {epoch}: {(n_correct*100)/nb_tr_examples}')
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Training Loss Epoch: {epoch_loss}")
    print(f"Training Accuracy Epoch: {epoch_accu}")

    return 

In [53]:
EPOCHS = 1
for epoch in range(EPOCHS):
    train(epoch)

0it [00:00, ?it/s]FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).


Training Loss per 5000 steps: 1.1217623949050903
Training Accuracy per 5000 steps: 37.5


5it [00:06,  1.34s/it]


KeyboardInterrupt: ignored